## Introduction

In this assignment, you are going to build a classifier for named entities from the Groningen Meaning Bank corpus.  Named entity recognition (NER) takes noun phrases from a text and identifies whether they are persons, organizations, and so on.  You will be using the Groningen Meaning Bank named entity corpus available on mltgpu at `/scratch/lt2222-v21-resources/GMB_dataset.txt`.  In this version of the task, you will assume we know *that* something is a named entity, and instead use multi-class classification to identify its type.  So you will be doing named entity classification but *not* recognition.

The data looks like this: 

```
3996    182.0   Nicole  NNP     B-per
3997    182.0   Ritchie NNP     I-per
3998    182.0   is      VBZ     O
3999    182.0   pregnant        JJ      O
4000    182.0   .       .       O
4001    183.0   Speaking        VBG     O
4002    183.0   to      TO      O
4003    183.0   ABC     NNP     B-org
4004    183.0   News    NNP     I-org
4005    183.0   interviewer     NN      O
4006    183.0   Dianne  NNP     B-per
4007    183.0   Sawyer  NNP     I-per
4008    183.0   ,       ,       O
4009    183.0   the     DT      O
4010    183.0   25-year-old     JJ      O
4011    183.0   co-star NN      O
4012    183.0   of      IN      O
4013    183.0   TV      NN      O
4014    183.0   's      POS     O
4015    183.0   The     DT      B-art
4016    183.0   Simple  NNP     I-art
4017    183.0   Life    NNP     I-art
4018    183.0   said    VBD     O
4019    183.0   she     PRP     O
4020    183.0   is      VBZ     O
4021    183.0   almost  RB      O
4022    183.0   four    CD      O
4023    183.0   months  NNS     O
4024    183.0   along   IN      O
4025    183.0   in      IN      O
4026    183.0   her     PRP$    O
4027    183.0   pregnancy       NN      O
4028    183.0   .       .       O
```

The first column is the line number.  The second column is a sentence number (for some reason given as a float; ignore it).  The third column is the word.  The fourth column is a part of speech (POS) tag in Penn Treebank format.  The last column contains the named entity annotation. 

The annotation works like this.  Every `O` just means that the row does not represent a named entity.  `B-xyx` means the first word in a named entity with type `xyx`. `I-xyz` means the second and later words of an `xyz` entity, if there are any.  That means that every time there's a `B` or an `I`, there's a named entity.  

The entity types in the corpus are `art`,
`eve`,
`geo`,
`gpe`,
`nat`,
`org`,
`per`,
and `tim`

Your task is the following.
1. To preprocess the text (lowercase and lemmatize; punctuation can be preserved as it gets its own rows).
2. To create instances from every from every identified named entity in the text with the type of the NE as the class, and a surrounding context of five words on either side as the features.  
3. To generate vectors and split the instances into training and testing datasets at random.
4. To train a support vector machine (via `sklearn.svm.LinearSVC`) for classifying the NERs.
5. To evaluate the performance of the classifier.

You will do this by modifying a separate file containing functions that will be called from this notebook as a module.  You can modify this notebook for testing purposes but please only submit the original.  You will document everything in Markdown in README.md and submit a GitHub repository URL.

This assignment is due on **Tuesday, 2021 March 9 at 23:59**.  It has **25 points** and **7 bonus points**.

In [1]:
import a2
from sklearn.svm import LinearSVC

In [2]:
gmbfile = open('GMB_dataset.txt', "r")
# (If running on MLT server)
# gmbfile = open('/scratch/lt2222-v21-resources/GMB_dataset.txt', "r")

## Part 1 - preprocessing (3 points)

See step 1 above.  The data is coming to you as an unused file handle object.  You can return the data in any indexable form you like.  You can also choose to remove infrequent or uninformative words to reduce the size of the feature space. (Document this in README.md.)

In [3]:
inputdata = a2.preprocess(gmbfile)
gmbfile.close()
inputdata[20:40]

,sentenceNr,word,POS,NEtag
20,1,from,IN,O
21,1,that,DT,O
22,1,country,NN,O
23,2,family,NNS,O
24,2,of,IN,O
25,2,soldier,NNS,O
26,2,kill,VBN,O
27,2,in,IN,O
28,2,the,DT,O
29,2,conflict,NN,O


## Part 2 - Creating instances (7 points)

Do step 2 above.  You will create a collection of Instance objects.  Remember to consider the case where the NE is at the beginning of a sentence or at the end, or close to either (you can create a special start token for that).  You can also start counting from before the `B` end of the NE mention and after the last `I` of the NE mention. That means that the instances should include things before and after the named entity mention, but not the named entity text itself.

In [6]:
instances = a2.create_instances(inputdata)

In [7]:
instances[20:40]

[Class: gpe Features: ['<s1>', '<s2>', '<s3>', '<s4>', '<s5>', 'this', 'week', 'restart', 'part', 'of'],
 Class: geo Features: ['the', 'conversion', 'process', 'at', 'its', 'nuclear', 'plant', '</s1>', '</s2>', '</s3>'],
 Class: gpe Features: ['<s1>', '<s2>', '<s3>', '<s4>', '<s5>', 'official', 'say', 'they', 'expect', 'to'],
 Class: tim Features: ['sensitive', 'part', 'of', 'the', 'plant', 'after', 'an', 'iaea', 'surveillance', 'system'],
 Class: org Features: ['the', 'plant', 'wednesday', 'after', 'an', 'surveillance', 'system', 'begin', 'function', '</s1>'],
 Class: org Features: ['<s2>', '<s3>', '<s4>', '<s5>', 'The', 'surveillance', 'system', 'begin', 'function', '</s1>'],
 Class: gpe Features: ['<s5>', 'The', 'european', 'union', 'with', 'backing', 'have', 'threaten', 'to', 'refer'],
 Class: gpe Features: ['backing', 'have', 'threaten', 'to', 'refer', 'to', 'the', 'u.n.', 'security', 'council'],
 Class: org Features: ['to', 'refer', 'iran', 'to', 'the', 'to', 'the', 'u.n.', 'secu

## Part 3 - Creating the table and splitting (10 points)

Here you're going to write the functions that create a data table with "document" vectors representing each instance and split the table into training and testing sets and random with an 80%/20% train/test split.

In [8]:
bigdf = a2.create_table(instances)
bigdf[20:40]

,NE_class,vector
20,gpe,"[0.2918489898787147, -0.2228923629539286, -0.0..."
21,geo,"[0.15747443330560373, 0.14620880871082087, 0.0..."
22,gpe,"[0.400964755379122, -0.3588337570380066, -0.01..."
23,tim,"[0.04316814481269091, 0.012291577274904292, 0...."
24,org,"[0.06821698262936311, 0.04101560676950451, 0.0..."
25,org,"[0.24514571016546285, -0.12948057912298272, -0..."
26,gpe,"[0.09509039751821698, -0.06196937321963152, 0...."
27,gpe,"[0.06741957692589473, -0.014630341345356138, 0..."
28,org,"[0.0900105570171458, 0.007763167024603573, 0.2..."
29,gpe,"[0.04254628685586361, -0.020090041525577085, 0..."


In [9]:
train_X, train_y, test_X, test_y = a2.ttsplit(bigdf)

# X and y mean feature matrix and class respectively.
train_X, train_y, test_X, test_y

(array([[ 0.0900106 , -0.08598886,  0.12926705, ...,  0.0022298 ,
          0.00179209,  0.00265109],
        [ 0.2543241 ,  0.30026571, -0.04386027, ..., -0.00096746,
          0.00088428, -0.00735862],
        [ 0.0296561 ,  0.00051986,  0.07662743, ...,  0.02975004,
         -0.01148424,  0.01464954],
        ...,
        [ 0.06889689,  0.01556479,  0.17568195, ..., -0.00245458,
          0.00150949, -0.0067574 ],
        [ 0.0564334 , -0.01593151,  0.10847073, ...,  0.00341775,
          0.00203227, -0.00264359],
        [ 0.20090732, -0.13070457,  0.10478341, ...,  0.00358857,
          0.00185618,  0.00227532]]),
 1019    per
 4333    tim
 6026    per
 5466    org
 5060    geo
        ... 
 6781    tim
 6833    per
 370     org
 2575    tim
 2542    tim
 Name: NE_class, Length: 5538, dtype: object,
 array([[ 0.09170924,  0.02798102,  0.23018494, ..., -0.00436365,
          0.0030632 ,  0.00391933],
        [ 0.24271281, -0.17949593, -0.00728888, ..., -0.00121682,
          0.0052

In [10]:
len(test_y) / (len(test_y) + len(train_y))

0.1999422132331696

In [11]:
len(test_X) / (len(test_X) + len(train_X))

0.1999422132331696

In [12]:
# test_y[0] # *key error if 0th row not in test dada
zeroth = test_y[0] if 0 in test_y else train_y[0]
zeroth

'geo'

## Part 4 - Training the model (0 points)

This part you won't do yourself.

In [13]:
model = LinearSVC()
model.fit(train_X, train_y)
train_predictions = model.predict(train_X)
test_predictions = model.predict(test_X)

In [14]:
train_predictions

array(['tim', 'tim', 'per', ..., 'org', 'gpe', 'tim'], dtype=object)

In [15]:
train_y

1019    per
4333    tim
6026    per
5466    org
5060    geo
       ... 
6781    tim
6833    per
370     org
2575    tim
2542    tim
Name: NE_class, Length: 5538, dtype: object

In [16]:
test_predictions

array(['org', 'per', 'geo', ..., 'gpe', 'per', 'per'], dtype=object)

In [17]:
test_y

5       geo
12      geo
18      tim
19      gpe
24      org
       ... 
6901    org
6908    geo
6909    geo
6913    tim
6915    per
Name: NE_class, Length: 1384, dtype: object

## Part 5 - Evaluation (5 points)

Investigate for yourself what a "confusion matrix".  Then implement a function that takes the data and produces a confusion matrix in any readable form that allows us to compare the performance of the model by class.  

In [18]:
test_y #1384 'gold' NE classes 

5       geo
12      geo
18      tim
19      gpe
24      org
       ... 
6901    org
6908    geo
6909    geo
6913    tim
6915    per
Name: NE_class, Length: 1384, dtype: object

In [19]:
test_predictions #1384 preducted NE classes

# confusion matrix: N*N ie 8*8

array(['org', 'per', 'geo', ..., 'gpe', 'per', 'per'], dtype=object)

In [20]:
a2.confusion_matrix(test_y, test_predictions)

,art,eve,geo,gpe,nat,org,per,tim
art,1,0,1,1,0,0,1,2
eve,0,0,0,1,0,0,1,1
geo,1,3,232,76,0,62,45,71
gpe,1,0,50,78,1,34,45,17
nat,0,0,1,0,2,0,0,0
org,1,0,52,37,0,79,32,33
per,0,1,24,47,0,39,81,19
tim,2,0,55,18,1,29,16,90


In [21]:
a2.confusion_matrix(train_y, train_predictions)

,art,eve,geo,gpe,nat,org,per,tim
art,44,0,0,0,0,0,1,0
eve,0,36,1,0,0,0,0,0
geo,1,1,1491,55,1,81,44,81
gpe,0,0,41,813,0,38,36,18
nat,0,0,0,0,15,0,0,0
org,2,0,47,35,0,806,17,23
per,0,3,42,52,0,46,768,22
tim,0,1,33,17,0,23,20,783


Examine the matrix and describe your observations in README.md.  In particular, what do you notice about the predictions on the training data compared to those on the test data.

## Bonus Part A - Error analysis (2 points)

Look at the weakest-performing classes in the confusion matrix (or any, if they all perform poorly to the same extent).  Find some examples in the test data on which the classifier classified incorrectly for those classes.  What do you think is the reason why those are hard?  Consider linguistic factors and statistical factors, if applicable.  Write your answer in README.md.

## Bonus Part B - Expanding the feature space (7 points)

Run the entire process above, but incorporate part-of-speech tag information into the feature vectors.  It's your choice as to how to do this, but document it in README.md.  Your new process should run from the single call below:

In [20]:
a2.bonusb('GMB_dataset.txt')

In [21]:
import spacy
from tqdm import tqdm
with open('GMB_dataset.txt', "r") as file:
    # Initialize spacy 'en' model, keeping only tagger component needed for lemmatization
    nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
    
    sentences_dict = {}
    for line in tqdm(file.readlines()[1:], desc='Adding each word: '):
        line=line.strip('\n')
        items = line.split('\t')  # [indexNr, sentenceNr, Word,	POS, NEtag]
        items[1] = int( float(items[1]) )  # Turn sentenceNr "NNN.0" into integer
        items[2] = ' '.join([w.lemma_ for w in en_nlp( items[2] )]).lower() # Use Spacy to lemmatise & lowercase
        items[4] = False if items[4]=='O' else items[4]
        sentNr, word, POS, NEclass=items[1:]
        
        if sentNr not in sentences_dict:
            sentences_dict[sentNr]=[]
        sentences_dict[sentNr].append( (word, POS, NEclass) )

sentences_dict[189]

Adding each word:   0%|                                                                      | 0/66161 [00:00<?, ?it/s]


NameError: name 'en_nlp' is not defined

In [ ]:
sentences_dict[189]

In [ ]:
NEs_list=[]
for nr, sent in sentences_dict.items():
    for trio in sent:
        word,POS,NEclass=trio[:]
        if NEclass and NEclass[0]=='B':
            words=[word]
            i=sent.index(trio)
            c=1
            try:
                
                while sent[i+c][2]:
                    words.append(sent[i+c][0])
                    c+=1
                    
                if sent[i+c][2]==False:
                    next5=[]
                    for nxt in range(5):
                        try:
                            nextword=sent[i+c+nxt][0]
                            next5.append(nextword)
                        except IndexError:
                            pass
                        
            except IndexError:
                pass
            
            prev5=[]
            for prev in range(5,0,-1):
                try:
                    if i-prev>=0:
                        prevword=sent[i-prev][0]
                        prev5.append(prevword)
                except IndexError:
                    pass
                
                
            NE = ' '.join(words)
            NEs_list.append((NE,NEclass[2:],prev5, next5))
                
            
NEs_list  


In [ ]:
len(NEs_list)